In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
from scipy.stats import norm

# ==========================
# 1. Definir portafolios
# ==========================
tickers = [
    "GLD", "SHY", "TIP", "NVDA", "KO",
    "BTC-USD", "JPM", "AAPL", "IEF", "TSLA"
]

weights_P2 = np.array([
    0.2500,  # GLD
    0.2500,  # SHY
    0.2029,  # TIP
    0.1280,  # NVDA
    0.0833,  # KO
    0.0275,  # BTC-USD
    0.0211,  # JPM
    0.0199,  # AAPL
    0.0113,  # IEF
    0.0061   # TSLA
])

weights_P1 = np.array([
    0.2500,  # GLD
    0.2500,  # SHY
    0.1998,  # TIP
    0.1551,  # NVDA
    0.0810,  # KO
    0.0260,  # BTC-USD
    0.0190,  # JPM
    0.0050,  # AAPL
    0.0000,  # IEF (no está en P1)
    0.0140   # TSLA
])

# Normalizar por si acaso
weights_P1 = weights_P1 / weights_P1.sum()
weights_P2 = weights_P2 / weights_P2.sum()

# ==========================
# 2. Descargar precios
# ==========================
START = "2020-09-01"   # cambia la fecha si quieres
END   = None           # hasta hoy

extra_tickers = ["SPY", "TBIL"]  # benchmark y "tasa libre de riesgo"
all_tickers = tickers + extra_tickers

data = yf.download(all_tickers, start=START, end=END)["Close"].dropna()

# returns simples diarios
rets = data.pct_change().dropna()

# separar series
rets_assets = rets[tickers]
r_bench = rets["SPY"]
r_rf    = rets["TBIL"]  # proxy libre de riesgo; también podrías usar un promedio constante

# ==========================
# 3. Funciones de métricas
# ==========================
TRADING_DAYS = 252

def portfolio_returns(weights, asset_returns):
    """Serie de retornos diarios del portafolio."""
    return (asset_returns @ weights).rename("Rp")

def annualized_return(r):
    mu_d = r.mean()
    return mu_d * TRADING_DAYS

def annualized_vol(r):
    sigma_d = r.std()
    return sigma_d * np.sqrt(TRADING_DAYS)

def sharpe_ratio(rp, r_rf):
    """Sharpe anualizado (usa diferencia diaria vs TBIL)."""
    excess = rp - r_rf
    mu_excess = excess.mean() * TRADING_DAYS
    sigma_excess = excess.std() * np.sqrt(TRADING_DAYS)
    return mu_excess / sigma_excess

def beta_vs_bench(rp, rb):
    cov = np.cov(rp, rb)[0,1]
    var_b = np.var(rb)
    return cov / var_b

def treynor_ratio(rp, r_rf, rb):
    """Treynor anualizado."""
    beta = beta_vs_bench(rp, rb)
    mu_p = annualized_return(rp)
    mu_rf = annualized_return(r_rf)
    return (mu_p - mu_rf) / beta, beta

def var_historical(rp, alpha=0.95):
    """VaR histórico diario (retorno, signo negativo = pérdida)."""
    return np.quantile(rp, 1 - alpha)

def cvar_historical(rp, alpha=0.95):
    """CVaR / Expected Shortfall diario."""
    var_level = var_historical(rp, alpha)
    tail = rp[rp <= var_level]
    return tail.mean()

def var_parametric(rp, alpha=0.95):
    """VaR paramétrico diario (supuesto normal)."""
    mu = rp.mean()
    sigma = rp.std()
    z = norm.ppf(1 - alpha)  # negativo
    return mu + z * sigma

def max_drawdown_and_duration(rp):
    """Máx. drawdown y duración (en días)."""
    # curva de valor 1 + retorno acumulado
    cum = (1 + rp).cumprod()
    running_max = cum.cummax()
    drawdown = cum / running_max - 1.0

    max_dd = drawdown.min()

    # duración: racha más larga con drawdown < 0
    dd_flag = drawdown < 0
    max_dur = 0
    current = 0
    for in_dd in dd_flag:
        if in_dd:
            current += 1
            max_dur = max(max_dur, current)
        else:
            current = 0

    return max_dd, max_dur

def summarize_portfolio(name, weights):
    rp = portfolio_returns(weights, rets_assets)

    mu_ann  = annualized_return(rp)
    vol_ann = annualized_vol(rp)
    sharpe  = sharpe_ratio(rp, r_rf)
    treynor, beta = treynor_ratio(rp, r_rf, r_bench)

    var_hist = var_historical(rp, alpha=0.95)
    cvar_hist = cvar_historical(rp, alpha=0.95)
    var_param = var_parametric(rp, alpha=0.95)

    max_dd, dur_dd = max_drawdown_and_duration(rp)

    print(f"\n===== {name} =====")
    print(f"Retorno anualizado       : {mu_ann:.4%}")
    print(f"Volatilidad anualizada   : {vol_ann:.4%}")
    print(f"Sharpe (TBIL)            : {sharpe:.3f}")
    print(f"Beta vs SPY              : {beta:.3f}")
    print(f"Treynor (SPY)            : {treynor:.3f}")
    print(f"VaR hist 95% (diario)    : {var_hist:.4%}")
    print(f"CVaR hist 95% (diario)   : {cvar_hist:.4%}")
    print(f"VaR param 95% (diario)   : {var_param:.4%}")
    print(f"Máx drawdown             : {max_dd:.2%}")
    print(f"Duración máx drawdown    : {dur_dd} días")

# ==========================
# 4. Ejecutar para P1 y P2
# ==========================
summarize_portfolio("Portafolio 1 (P1)", weights_P1)
summarize_portfolio("Portafolio 2 (P2)", weights_P2)


/tmp/ipython-input-3684362157.py:53: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(all_tickers, start=START, end=END)["Close"].dropna()
[*********************100%***********************]  12 of 12 completed



===== Portafolio 1 (P1) =====
Retorno anualizado       : 24.2642%
Volatilidad anualizada   : 11.0106%
Sharpe (TBIL)            : 1.792
Beta vs SPY              : 0.480
Treynor (SPY)            : 0.410
VaR hist 95% (diario)    : -1.0195%
CVaR hist 95% (diario)   : -1.4034%
VaR param 95% (diario)   : -1.0446%
Máx drawdown             : -13.40%
Duración máx drawdown    : 109 días

===== Portafolio 2 (P2) =====
Retorno anualizado       : 22.2161%
Volatilidad anualizada   : 9.9297%
Sharpe (TBIL)            : 1.781
Beta vs SPY              : 0.430
Treynor (SPY)            : 0.410
VaR hist 95% (diario)    : -0.8995%
CVaR hist 95% (diario)   : -1.2616%
VaR param 95% (diario)   : -0.9407%
Máx drawdown             : -12.39%
Duración máx drawdown    : 109 días
